# Imports

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
df = pd.read_csv("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/20230611-combined.csv", index_col=0)

# Design

The purpose of this notebook is to outline the different options for possible ways of splitting up the structures

The idea is that each section will describe how to take an input set of c

# Random

# Ligand-based : only the identity of the ligand matters

# Structure-based : only the identity of the structure matters

## n structure per date?

## aggregated by month

In [ ]:
df["Structure_Date"] = df.Structure_Date.apply(lambda x: x[:-3])
by_date = df.groupby(["Version", "Compound_ID", "Structure_Date"])["Docked_File"].count().reset_index()

In [ ]:
by_date_group = by_date.groupby(["Version", "Structure_Date"])["Docked_File"]
by_date_agg = by_date_group.mean().reset_index()
by_date_agg["Std"] = by_date_group.std().reset_index()["Docked_File"]

In [ ]:
by_date_agg

In [ ]:
px.bar(by_date_agg, color="Version", x="Structure_Date", y="Docked_File", barmode='group', error_y="Std")

## cumulative

In [ ]:
px.histogram(by_date_agg, x="Structure_Date", y="Docked_File", color="Version",cumulative=True,barmode='group')

## cumulative with errors

In [ ]:
df = pd.read_csv("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/20230611-combined.csv", index_col=0)

In [ ]:
df["Structure_Date"] = df.Structure_Date.apply(lambda x: x[:-3])
by_date = df.groupby(["Version", "Compound_ID", "Structure_Date"])["Docked_File"].count().reset_index()

In [ ]:
by_date

In [ ]:
by_date_by_cmpd = by_date.groupby(["Version", "Compound_ID"]).apply(lambda x: x["Docked_File"].cumsum()).reset_index().drop(columns=["level_2"])

In [ ]:
by_date_by_cmpd["Structure_Date"] = by_date.Structure_Date

In [ ]:
px.line(by_date_by_cmpd, color="Compound_ID", y="Docked_File", facet_col="Version", x="Structure_Date")

### combining with std

In [ ]:
grouped = by_date_by_cmpd.drop(columns="Compound_ID").groupby(["Version", "Structure_Date"])

In [ ]:
with_error = grouped.mean().reset_index()

In [ ]:
with_error["Error_minus"] = with_error.Docked_File - grouped.min().reset_index().Docked_File
with_error["Error_plus"] = grouped.max().reset_index().Docked_File - with_error.Docked_File

In [ ]:
px.bar(with_error, x="Structure_Date", y="Docked_File", color="Version",barmode='group', 
       error_y_minus="Error_minus", error_y="Error_plus"
      )

# Ligand + Structure : Ligand Similarity

# Ligand + Structure : Active Site Similarity